## 1. Realizamos el login en la API REST

In [26]:
import logging
from json import load
# Importing models and REST client class from Professional Edition version
from tb_rest_client.rest_client_pe import *
from tb_rest_client.rest import ApiException


logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(module)s - %(lineno)d - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')


# ThingsBoard REST API URL
url = "https://development.akiot.es/"

# Default Tenant Administrator credentials
username = "tenant@akiot.es"
password = "tenant"


## 2. Nos autenticamos y obtenemos el actual usuario

In [27]:
with RestClientPE(base_url=url) as rest_client:
    try:
        # Auth with credentials
        rest_client.login(username=username, password=password)

        # Getting current user
        current_user = rest_client.get_user()
        logging.info('Current user:\n%r\n', current_user)

    except ApiException as e:
            logging.exception(e)

2025-11-11 17:19:16 - INFO - 470615400 - 8 - Current user:
{'additional_info': {'homeDashboardHideToolbar': True,
                     'lastLoginTs': None,
                     'userActivated': True,
                     'userCredentialsEnabled': True},
 'authority': 'TENANT_ADMIN',
 'created_time': 1760987466533,
 'custom_menu_id': None,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '13814000-1dd2-11b2-8080-808080808080'},
 'email': 'tenant@akiot.es',
 'first_name': None,
 'id': {'entity_type': 'USER', 'id': '87789550-ade8-11f0-b1bc-33c092a70dea'},
 'last_name': None,
 'name': 'tenant@akiot.es',
 'owner_id': {'entity_type': 'TENANT',
              'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'phone': None,
 'tenant_id': {'entity_type': 'TENANT',
               'id': '86c746b0-ade8-11f0-b1bc-33c092a70dea'},
 'version': 2}



## 3. Creamos Grupos para distintas entidades

In [28]:
# Creo un GRUPO de activos en el nivel de tenant: "farmacias_cofa"
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "ASSET"
group_name = "farmacias_cofa"

try:
    # 1.1 Intentar obtener perfiles de activos para verificar si ya existe
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
    
except ApiException as e:
    if e.status == 404:
        import json
        error_body = json.loads(e.body)
        if error_body.get('message') == "Requested item wasn't found!":
            # Si se encuentra el error esperado 404:
            print("El grupo no fue encontrado (404). Puedes proceder a crearlo.")
            # Creamos el grupo
            shared_asset_group = EntityGroup(name=group_name, type=group_type)
            shared_asset_group = rest_client.save_entity_group(shared_asset_group)
            logging.info('Asset group created:\n%r\n', shared_asset_group)
        else:
            # Otro error 404 (mensaje inesperado)
            raise e # Lanza de nuevo la excepción para manejarla más arriba









2025-11-11 17:19:18 - INFO - 2330404645 - 10 - Buscando grupo de ASSET con nombre 'farmacias_cofa' para el propietario '86c746b0-ade8-11f0-b1bc-33c092a70dea'...
2025-11-11 17:19:21 - INFO - 2330404645 - 16 - El grupo de ASSET 'farmacias_cofa' ya existe.


In [29]:
# Creamos Grupo de Customers "COFA" a nivel de Tenant
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "CUSTOMER"
group_name = "COFA"

try:
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    shared_customer_group = existing_group
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
    
except ApiException as e:
    if e.status == 404:
        import json
        error_body = json.loads(e.body)
        if error_body.get('message') == "Requested item wasn't found!":
            # Si se encuentra el error esperado 404:
            print("El grupo no fue encontrado (404). Puedes proceder a crearlo.")
            # Creamos el grupo
            shared_customer_group = EntityGroup(name=group_name, type=group_type)
            shared_customer_group = rest_client.save_entity_group(shared_customer_group)
            logging.info('Asset group created:\n%r\n', shared_customer_group)
        else:
            # Otro error 404 (mensaje inesperado)
            raise e # Lanza de nuevo la excepción para manejarla más arriba

2025-11-11 17:19:23 - INFO - 242566287 - 9 - Buscando grupo de CUSTOMER con nombre 'COFA' para el propietario '86c746b0-ade8-11f0-b1bc-33c092a70dea'...
2025-11-11 17:19:23 - INFO - 242566287 - 16 - El grupo de CUSTOMER 'COFA' ya existe.


In [30]:
# Creo un perfil para los activos "Farmacias COFA"
from tb_rest_client.models.models_pe import EntityId
import logging
from pprint import pprint
profile_name = "Farmacias COFA"
profile_description = "Default asset profile"

# asset_profile = AssetProfile(
#         name=profile_name ,
#         description=profile_description,
#         # Otros campos opcionales:
#         # default_queue_name="Main" 
#         # default=False # No marcar como perfil por defecto
#     )

# saved_asset_profile = rest_client.save_asset_profile(asset_profile)
try:
    # 1.1 Intentar obtener perfiles de activos para verificar si ya existe
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    page_data = rest_client.get_asset_profiles(page_size=100, page=0, text_search=profile_name)

    existing_profile = next((profile for profile in page_data.data if profile.name == profile_name), None)

    if existing_profile:
        logging.info(f"El perfil de activo '{profile_name}' ya existe!!.")
        profile_asset_to_use = existing_profile.id
        raise Exception(f"El perfil de activo '{profile_name}' ya existe!!.")
    else:
    # 1.2 Crear una instancia del modelo AssetProfile
        new_asset_profile = AssetProfile(
                                        name=profile_name,
                                        description=profile_description,)

    # 1.3 Guardar el perfil de activo utilizando el método save_asset_profile
        created_profile = rest_client.save_asset_profile(body=new_asset_profile)

        profile_asset_to_use = created_profile.id
        logging.info(f"✅ Perfil de activo '{profile_name}' creado exitosamente.")
            
except Exception as e:
        logging.error(f"❌ Error al crear/obtener el perfil de activo: {e}")





2025-11-11 17:19:27 - INFO - 193593744 - 25 - El perfil de activo 'Farmacias COFA' ya existe!!.
2025-11-11 17:19:27 - ERROR - 193593744 - 41 - ❌ Error al crear/obtener el perfil de activo: El perfil de activo 'Farmacias COFA' ya existe!!.


## 4. Vamos a crear el Customer con sus atributos y crear el activo FARMACIA con sus atributos y se lo vamos a asignar
Por cada activo  (Farmacia) debería Asignar:
- Un Name del activo: Farmacias Garcia 
- Un Label: CUIT o identificador único
- Un Type, perfil de activo: Farmacias COFA
- Una Descripción Opcional
- Varias Farmacias pueden tener el mismo CUIT, entonces, se debe crear o buscar un customer filtrado por CUIT, asignarle el grupo COFA. Este Customer es independiente de las x cuentas de usuario que se puedan crear para acceso.
- Es necesario que por cada CUIT que puede tener asociado varias farmacias, se tenga un responsable de la/las farmacia/s asociadas a ese CUIT. Esto es independiente de los usuarios que acceden a la plataforma. De hecho pueden ser otros personas distintas al responsable de farmacias.
- Luego se asigna el activo con identificador único para cada farmacia a ese customer (CUIT de farmacia).

- Se podría crear un grupo de activo, pero en este caso no haría falta porque cada cliente tiene asignado su dispositivo

### Ejemplo de fila de datos en el CSV


| Name | Type | Label | ID\_Farmacia | calleNumero | ciudad | provincia | codigoPostal | latitud | longitud | responsable | cargo | correo | CUIT\_Farmacia | celular | chat\_id | token\_telegram |
|:---|:---|:---|:---:|:---|:---|:---|:---|:---:|:---:|:---|:---|:---|:---|:---|:---:|:---|
| Farmacia DEMO 1 | Farmacias COFA | 1 | 1 | Av. Belgrano 869 | CABA | Buenos Aires | CP1097 | -34.6124 | -58.3787 | Fulano Peña | Dueño | fpenia@gmail.com | 20-44267315-5 | 549 1177000200 | -12345 | 8491771441:AAFneI93qwYSB\_5Rz-WQEB0z6HIlKheWp\_c |
| Farmacia DEMO 2 | Farmacias COFA | 2 | 2 | Av. Paseo Colón 850 | CABA | Buenos Aires | C1063ACV | -34.7747 | -58.7253 | Juan Corvalan | Gerente | carencibia@gmail.com | 30-71234567-8 | 549 1125846546 | 1234 | 8491771879:AAFneI93qwYSB\_5Rz-WQEB0z6HIlKheWp\_c |
| Farmacia DEMO 3 | Farmacias COFA | 2 | 3 | Calle 53 | La Plata | Buenos Aires | B1900TVA | -34.9214 | -58.3544 | Pablo Heredia | Dueño | pheredia@gmail.com | 20-44267315-5 | 549 2215551234 | -56485 | 7891771441:AAFneI93qwYSB\_5Rz-WQEB0z6HIlKheWp\_c |

## ¡IMPORTANTE!, Si borro el Customer se borra toda sus entidades, por ejemplo se borrarían todos sus users, y se perdería la relación con sus activos y dispositivos
Por esta razón se lo debe crear con el siguiente script, con todos sus atributos, users, dispositivos, activos, etc. En un futuro si se quiere modificar sus entidades se hara atraves de la appWeb o se crear otro script que lea el customer y modifique lo necesario.

Voy a crear los customers con los CUIT de farmacia, debe tener un chat_id, un token de telegram y un correo asociado a ese CUIT.
Luego ese CUIT (customer) tiene Farmacias(activos), Sensores(dispositivos).


In [31]:
# Creamos, el customer y lo asociamos a su grupo "COFA", luego creamos el activo para cada fila del CSV, luego establecemos la relación entre ambos.
import csv
from utils import creating_asset
file_path = 'activos_COFA_ejemplo.csv'

try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                name = datos.get("Name", "")
                tipo = datos.get("Type", "")
                label = datos.get("Label", "")
                id_farmacia = datos.get("ID_farmacia", "")
                calleNumero = datos.get("calleNumero", "")
                ciudad = datos.get("ciudad", "")
                provincia = datos.get("provincia", "")
                codigoPostal = datos.get("codigoPostal", "")
                latitud = datos.get("latitud", "")
                longitud = datos.get("longitud", "")
                cuit_farmacia = datos.get("CUIT_farmacia", "")
                responsable = datos.get("responsable_farmacias", "")
                cargo = datos.get("cargo", "")
                correo = datos.get("correo", "")
                celular = datos.get("celular", "")
                chat_id = datos.get("chat_id", "")
                token_telegram = datos.get("token_telegram", "")

                # Intentamos crear el customers
                try:
                    # 1.1 Intentar obtener todos los customers para verificar si ya existe
                    # page_data = rest_client.get_customers(page_size=100, page=0, text_search="CP1097")
                    page_data = rest_client.get_customers(page_size=1000, page=0)

                    existing_profile_customer = next((profile for profile in page_data.data if profile.address2  == cuit_farmacia), None)
                    # existing_profile_customer = next((profile for profile in page_data.data if profile.email == correo), None)

                    if existing_profile_customer:
                        # logging.info(f"El customer {responsable} ya existe !!, el CUIT asociado es {cuit} y su correo es  {correo} ")

                        # SERIA MEJOR FILTRAR POR DNI DEL RESPONSABLE, SOLICITAR QUE LO AGREGUEN AL CSV
                        logging.info(f"El customer con CUIT {cuit_farmacia} ya existe !!")
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer ya existente
                        # Ver si el activo existe por id_farmacia, tal que el id_farmacia puede ser la etiqueta o un atributo del servidor.
                        # Luego se debe asignar el perfil creado para el activo Farmacias COFA
                        # En este caso se supone que el customer ya tiene sus atributos creados correctamente y solo se corre aquí para asignarle nuevos activos.
                                               
                        
                         # Creating an Asset

                        creating_asset( rest_client,
                                        name,
                                        id_farmacia,
                                        profile_asset_to_use,
                                        existing_profile_customer,
                                        calleNumero + ", " + ciudad + ", " + provincia + ", " + codigoPostal,
                                        latitud,longitud,cuit_farmacia, responsable, correo, celular)


                         
                        raise Exception(f"El perfil de activo '{profile_name}' ya existe!!.")
                    else:
                        # 1.2 Creamos el customer
                        customer1 = Customer(title=responsable,
                                            email=correo,
                                            country="Argentina", # Valor fijo
                                            state=provincia,
                                            city=ciudad,
                                            address=calleNumero,
                                            address2=cuit_farmacia,  # Uso address2 para guardar el CUIT de la Farmacia y buscar luego
                                            zip=codigoPostal,
                                            phone=celular,)
                                            
                        customer1 = rest_client.save_customer(body=customer1)
                        # Asocio el customer al grupo "COFA"
                        rest_client.add_entities_to_entity_group(shared_customer_group.id, [customer1.id.id])
                        logging.info(f'✅ Customer creado exitosamente: {customer1.title}')
                        # Creo los atributos en el customer
                        # # Veo los atributos del customer
                        # atributos_customer = rest_client.telemetry_controller.get_attributes_by_scope_using_get(
                        #                                                                                         entity_type="CUSTOMER",
                        #                                                                                         entity_id=existing_profile_customer.id,
                        #                                                                                         scope="SERVER_SCOPE",
                        #                                                                                         )

                        # 1.3 Cargo nuevos atributos al customer
                        body_to_attributes = {
                            "cuit_farmacia": cuit_farmacia,
                            "chat_id": chat_id,
                            "token_telegram": token_telegram,
                            "email": correo # Se usa para las alarmas.

                        }
                        # Guardo un nuevo atributo de servidor para este customer
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="CUSTOMER",
                                entity_id=customer1.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)
                        
                        # 1.4 Comparto grupo de dashboard con el customer nuevo creado. Se supone que si ya lo cree ya tiene compartido el Dashboard.
                        page_data = rest_client.get_roles(page_size=1000, page=0,text_search="Entity Group Read-only User")
                        name_rol = page_data.data[0]
                        customer1_users = rest_client.get_entity_group_by_owner_and_name_and_type(customer1.id, "USER", "Customer Users")

                        # Buscamos el grupo de dashboards compartidos Farmacias COFA (a cada Customer luego de crearlo le comparto el Dashboard)
                        current_user = rest_client.get_user()
                        owner_id = current_user.tenant_id
                        group_type = "DASHBOARD"
                        group_name = "Farmacias COFA"

                        try:
                            logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
                            existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
                                    owner_id=owner_id, 
                                    group_type=group_type, 
                                    group_name=group_name
                                )
                            shared_dashboards_group = existing_group
                            logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
                        except ApiException as e:
                                    logging.error(f"❌ Error al buscar el grupo de dashboards: {e}")
                                    logging.exception(e)


                        # Assigning Shared Dashboards to the Customer 1 Users
                        tenant_id = current_user.tenant_id
                        group_permission = GroupPermission(role_id=name_rol.id,
                                                        name="",
                                                        user_group_id=customer1_users.id,
                                                        tenant_id=tenant_id,
                                                        entity_group_id=shared_dashboards_group.id,
                                                        entity_group_type=shared_dashboards_group.type)
                        group_permission = rest_client.save_group_permission(group_permission)
                        logging.info('Group permission created:\n%r\n', group_permission)
                        
                        # 1.6 Creating an Asset

                        creating_asset( rest_client,
                                        name,
                                        id_farmacia,
                                        profile_asset_to_use,
                                        customer1,
                                        calleNumero + ", " + ciudad + ", " + provincia + ", " + codigoPostal,
                                        latitud,longitud,cuit_farmacia, responsable, correo, celular)
                        
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el customer: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")


2025-11-11 17:19:32 - INFO - 1257833162 - 47 - El customer con CUIT 20-44267315-5 ya existe !!
2025-11-11 17:19:33 - ERROR - 1257833162 - 159 - Error al intentar crear el customer: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Length': '183', 'Content-Type': 'application/json;charset=ISO-8859-1', 'Date': 'Tue, 11 Nov 2025 17:19:32 GMT', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers'})
HTTP response body: b'{"status":400,"message":"Asset with such name already exists!","errorCode":31,"timestamp":1762881572730,"subscriptionErrorCode":null,"subscriptionEntry":null,"subscriptionValue":null}'

2025-11-11 17:19:33 - INFO - 1257833162 - 47 - El customer con CUIT 30-71234567-8 ya existe !!
2025-11-11 17:19:33 - ERROR - 1257833162 - 159 - Error al intentar crear el customer: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Content-Length': '183', 'Content-Type': 'application/json;charset=ISO-8859-1', 'Date': 'Tue, 11 

Hay que pensar en dispositos, crearlos a partir del excel que los relaciona con el customer(CUIT), Lo creo junto con el customer o si ya existe en customer, solo creo el dispo junto con activo. La otra es tener en otro documento la lista de customers(CUITS), la lista de activos (Farmacias) y una lista con dispositivos asignados, esta ultima nosotros lo asignamos como nos place. Ahi le asigno el dispositivo a el customer, pero lo relaciono solo con su Farmacia id_farma. tambien hay que tener en cuenta los atributos de servidor de los dispositivos. 
Por otro lado hay que pensar si en este nuevo script se cargan los usuarios o en el anterior a cada customer.

## 5. Carga de dispositivos
Los pasos serian:
Para cada dispositivo los pasos serian:
- Para cada dispositivo leer a que farmacia pertenece (activo relacionado)
- A partir de la farmacia obtener
    - La dirección de la misma (Dirección de la farmacia, atributo)
    - Customer esta asociado(Leer CUIT)
- Crear el dispositivo
- Cargar datos y atributos del server
- Asignar la propiedad del dispositivo al customer
- Relacionar hacia el dispositivo a la farmacia (activo)


In [ ]:
# Vamos a crear los dispositivos asociados a cada activo (Farmacia) y customer (CUIT)
# Si ya existe el dispositivo vamos a listar a quien ya pertenece.
import csv
from utils import creating_asset
file_path = 'dispositivos.csv'
name_device_profile = "HC5 NBIoT"
try:
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        lector = csv.DictReader(csvfile)  # Usa la primera fila como encabezados

        for fila in lector:
            try:
                # Cada fila es un diccionario
                datos = fila

                # Acceder a los valores por nombre de columna
                name = datos.get("name", "")
                tipo = datos.get("type", "")
                label = datos.get("label", "")
                token_acceso = datos.get("token_acceso", "")
                punto_monitoreo = datos.get("punto_monitoreo", "")
                relacion_farmacia = datos.get("relacion", "")

                # La dirección y el customer se obtienen de la farmacia(Activo)
                # address = datos.get("address", "")
                # customer = datos.get("customer", "")
                # Intentamos crear el customers
                try:
                    # 1.1 Intentar obtener el dispositivo por su nombre, ya que debe ser unico en AKIoT
                    
                    page_data = rest_client.get_user_devices(page_size=100, page=0, text_search=name)

                    if page_data.data==[]:
                        logging.info(f"El dispositivo con nombre '{name}' no existe, se puede crear.")
                        # busco el perfil de dispositivo ya creado en AKIoT

                        device_profile = rest_client.device_profile_controller.get_device_profile_infos_using_get(page_size=100, page=0, text_search=name_device_profile)
                        if not device_profile.data:
                            raise Exception(f"No se encontró el perfil de dispositivo con nombre '{name_device_profile}'")
                        
                        
                        
                        asset_to_relate = rest_client.asset_controller.get_all_asset_infos_using_get(page_size=100, page=0,include_customers=True, text_search=relacion_farmacia)
                        # Tengo el customer al que asignare este dispositivo
                        customer_relate_id = asset_to_relate.data[0].customer_id
                        # Tengo la entidad a la que relacionare este dispositivo
                        new_device = Device(name=name, label=label, device_profile_id=device_profile.data[0].id,customer_id=customer_relate_id)
                        new_device = rest_client.save_device(new_device,access_token=token_acceso)
                        logging.info(" Device was created:\n%r\n", new_device)



                        asset_to_relate_id = asset_to_relate.data[0]
                        attributes_from_asset = rest_client.telemetry_controller.get_attributes_by_scope_using_get(
                                                                                            entity_type="ASSET",
                                                                                            entity_id=asset_to_relate_id.id,
                                                                                            scope="SERVER_SCOPE",
                                                                                            )
                        attributes_from_asset = {item['key']: item['value'] for item in attributes_from_asset}
                        # Ya tengo los atributos del dispositivo, address, punto_monitoreo, xPos,yPos
                        body_to_attributes = {
                            "address": attributes_from_asset['direccion'],
                            "punto_monitoreo": punto_monitoreo,
                            # "xPos": attributes_from_asset['latitud'],
                            # "yPos": attributes_from_asset['longitud'],
                        }
                        # Guardo los atributos del servidor para este customer
                        try:
                            result = rest_client.telemetry_controller.save_entity_attributes_v1_using_post(
                                entity_type="DEVICE",
                                entity_id=new_device.id,
                                scope="SERVER_SCOPE",
                                body=body_to_attributes
                            )
                            print("✅ Atributo guardado correctamente:", result)
                        except ApiException as e:
                            print("❌ Error al guardar atributo:", e)
                        # Creamos la relacion entre el activo y el dispositivo
                        relation = EntityRelation(_from=asset_to_relate_id.id, to=new_device.id, type="Contains")
                        rest_client.save_relation(relation)
                        # Asignamos el dispositivo al customer (CUIT) correspondiente 
                        
                        # En los activos Farmacia que el title sea el CUIT, asi en la lista de dispositivo me queda lista de nombre de dispo CUIT al que pertenece, es practico para el buscador
                        # osea queremos customers por CUIT.


                    else:
                        # Si ya existe el dispositivo vamos a listar a quien ya pertenece.
                        customer_id= page_data.data[0].customer_id
                        try: 
                            customer_name = rest_client.get_customer_by_id(customer_id)
                            logging.info(f"Este dispositivo ya existe y pertenece a la farmacia con CUIT {customer_name.address2}")
        
                        except Exception as e:
                            raise Exception(f"El dispositivo no tiene customer, pertenece al tenant actual {e}")
                        
                        try:
                            # Buscamos su relacion con algun activo
                            relacion_from = rest_client.find_by_to_v1(page_data.data[0].id.id,
                                                                    'DEVICE',
                                                                    'CONTAINS')
                            activo = relacion_from[0]._from

                            activo = rest_client.asset_controller.get_asset_by_id_using_get(
                                    asset_id=activo.id,
                                )
                            
                            logging.info(f"Además el dispositivo esta relacionado con el activo: {activo.name}")
                            pass
                        except Exception as e:
                            raise Exception(f"El dispositivo no tiene activo relacionado {e}")
                                          
                        # Tomando los datos de la fila n, acá creo el activo y se lo asigno a ese customer recientemente creado
                except Exception as e:
                        logging.error(f"Error al intentar crear el dispositivo: {e}")
                

            except Exception as e:
                print(f"⚠️ Error procesando la fila: {fila}")
                print(f"   Detalle: {e}")

except FileNotFoundError:
    print("❌ No se encontró el archivo CSV.")
except Exception as e:
    print(f"❌ Error general al leer el archivo: {e}")

2025-11-11 19:08:51 - INFO - 2393964744 - 34 - El dispositivo con nombre 'AKIOT0F882' no existe, se puede crear.
2025-11-11 19:10:12 - INFO - 2393964744 - 49 -  Device was created:
{'additional_info': None,
 'created_time': 1762888153919,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': 'f7d9f880-be4a-11f0-9661-1da9e45f2bbc'},
 'device_data': {'configuration': {'type': 'DEFAULT'},
                 'transport_configuration': {'edrxCycle': None,
                                             'pagingTransmissionWindow': None,
                                             'powerMode': None,
                                             'psmActivityTimer': None,
                                             'type': 'COAP'}},
 'device_profile_id': {'entity_type': 'DEVICE_PROFILE',
                       'id': '8d64bdb0-b02a-11f0-9661-1da9e45f2bbc'},
 'firmware_id': None,
 'id': {'entity_type': 'DEVICE', 'id': 'e96f8cf0-bf31-11f0-9661-1da9e45f2bbc'},
 'label': 'Temperatura',
 'na

✅ Atributo guardado correctamente: b''


2025-11-11 19:10:17 - INFO - 2393964744 - 34 - El dispositivo con nombre 'AKIOT08AF3' no existe, se puede crear.
2025-11-11 19:10:23 - INFO - 2393964744 - 49 -  Device was created:
{'additional_info': None,
 'created_time': 1762888223641,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '73dfd940-be4b-11f0-9661-1da9e45f2bbc'},
 'device_data': {'configuration': {'type': 'DEFAULT'},
                 'transport_configuration': {'edrxCycle': None,
                                             'pagingTransmissionWindow': None,
                                             'powerMode': None,
                                             'psmActivityTimer': None,
                                             'type': 'COAP'}},
 'device_profile_id': {'entity_type': 'DEVICE_PROFILE',
                       'id': '8d64bdb0-b02a-11f0-9661-1da9e45f2bbc'},
 'firmware_id': None,
 'id': {'entity_type': 'DEVICE', 'id': '12fe4890-bf32-11f0-9661-1da9e45f2bbc'},
 'label': 'Temperatura',
 'na

✅ Atributo guardado correctamente: b''


2025-11-11 19:10:29 - INFO - 2393964744 - 34 - El dispositivo con nombre 'AKIOT08AF8' no existe, se puede crear.
2025-11-11 19:10:34 - INFO - 2393964744 - 49 -  Device was created:
{'additional_info': None,
 'created_time': 1762888234687,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': '73dfd940-be4b-11f0-9661-1da9e45f2bbc'},
 'device_data': {'configuration': {'type': 'DEFAULT'},
                 'transport_configuration': {'edrxCycle': None,
                                             'pagingTransmissionWindow': None,
                                             'powerMode': None,
                                             'psmActivityTimer': None,
                                             'type': 'COAP'}},
 'device_profile_id': {'entity_type': 'DEVICE_PROFILE',
                       'id': '8d64bdb0-b02a-11f0-9661-1da9e45f2bbc'},
 'firmware_id': None,
 'id': {'entity_type': 'DEVICE', 'id': '1993c4f0-bf32-11f0-9661-1da9e45f2bbc'},
 'label': 'Temperatura x 2',


✅ Atributo guardado correctamente: b''


2025-11-11 19:10:42 - INFO - 2393964744 - 34 - El dispositivo con nombre 'AKIOT08AF6' no existe, se puede crear.
2025-11-11 19:10:44 - INFO - 2393964744 - 49 -  Device was created:
{'additional_info': None,
 'created_time': 1762888244394,
 'customer_id': {'entity_type': 'CUSTOMER',
                 'id': 'f7d9f880-be4a-11f0-9661-1da9e45f2bbc'},
 'device_data': {'configuration': {'type': 'DEFAULT'},
                 'transport_configuration': {'edrxCycle': None,
                                             'pagingTransmissionWindow': None,
                                             'powerMode': None,
                                             'psmActivityTimer': None,
                                             'type': 'COAP'}},
 'device_profile_id': {'entity_type': 'DEVICE_PROFILE',
                       'id': '8d64bdb0-b02a-11f0-9661-1da9e45f2bbc'},
 'firmware_id': None,
 'id': {'entity_type': 'DEVICE', 'id': '1f5cf0a0-bf32-11f0-9661-1da9e45f2bbc'},
 'label': 'Temperatura',
 'na

✅ Atributo guardado correctamente: b''


In [ ]:
# Buscamos el grupo de dashboards compartidos Farmacias COFA (a cada Customer luego de crearlo le comparto el Dashboard)
current_user = rest_client.get_user()
owner_id = current_user.tenant_id
group_type = "DASHBOARD"
group_name = "Farmacias COFA"

try:
    # El método get_asset_profiles es paginado, por lo que buscamos en la primera página
    logging.info(f"Buscando grupo de {group_type} con nombre '{group_name}' para el propietario '{owner_id.id}'...")
    existing_group = rest_client.get_entity_group_by_owner_and_name_and_type(
            owner_id=owner_id, 
            group_type=group_type, 
            group_name=group_name
        )
    shared_dashboards_group = existing_group
    logging.info(f"El grupo de {group_type} '{group_name}' ya existe.")
except ApiException as e:
            logging.error(f"❌ Error al buscar el grupo de dashboards: {e}")
            logging.exception(e)


customer1_users = rest_client.get_entity_group_by_owner_and_name_and_type(customer1.id, "USER", "Customer Users")

# page_data = rest_client.get_roles(page_size=1000, page=0)
page_data = rest_client.get_roles(page_size=1000, page=0,text_search="Entity Group Read-only User")
pass

Vamos a crear dispositivos nombre, perfil (leer el perfil creado o crear uno), Label (Queda a dispo del cliente)
Luego los atributos ser servidor cargo: address(direction de la farmacia), tipoSensor(Importante, define el state Dashboard), xPos, yPos.
, y se lo asigno al customer correspondiente, esto es importante ya que si no le asigno el dispositivo no lo podrá ver. Luego ese dispositivo también se tiene que lo tiene que relacionar con la Farmacia correspondiente, el numero de serie/nombre del HC5 se asocia con el numero de CUIT de la Farmacia

Armar el CSV que vincule esto. 

